In [1]:
import os
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
import scipy

E:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


### Load dataset and split them into train and test set

In [2]:
filepath = 'C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/'
#####################   Modify above path   #########################
os.chdir(filepath)

# testpath=filepath+'/data/test_set/'
trainpath=filepath+'/data/train_set/'

# test_image_dir = testpath + "images/"
# test_pt_dir = testpath + "points/"
train_image_dir = trainpath + "images/"
train_pt_dir = trainpath + "points/"

import sklearn.metrics.pairwise
def pairwise_dist(vec):
    dist  = sklearn.metrics.pairwise_distances(vec, metric='euclidean')
    np.fill_diagonal(dist, np.nan)
    return dist
def feature(fiducial_pt_list,index):
    pairwise_dist_feature = pairwise_dist(fiducial_pt_list[index]).flatten()
    pairwise_dist_feature = pairwise_dist_feature[~np.isnan(pairwise_dist_feature)]
    return pairwise_dist_feature

f0 = time.time()
dataDir = train_pt_dir
fiducial_pt_list = []
filelist = []
for file in os.listdir(dataDir):
    filelist.append(file)
filelist.sort()
for file in filelist:
    fiducial_pt_list.append(scipy.io.loadmat(dataDir+file))
    l = []
for i in range(len(fiducial_pt_list)):
    if 'faceCoordinatesUnwarped' in fiducial_pt_list[i].keys():
        l.append(fiducial_pt_list[i]['faceCoordinatesUnwarped'])
    else:
        l.append(fiducial_pt_list[i]['faceCoordinates2'])
        
fiducial_pt_list = l


X = pd.DataFrame(np.zeros((2500, 6006)))
for i in range(2500):
    X.iloc[i,:] = np.round(feature(fiducial_pt_list, i).flatten(), 0)
y =pd.read_csv(trainpath+'label.csv')['emotion_idx']
f1 = time.time()-f0

scaler = StandardScaler()
X = scaler.fit_transform(X)

train_x_dis,test_x_dis,train_y_dis,test_y_dis=train_test_split(X,y,test_size=0.2,random_state=3662)
X_dis=X
y_dis=y

### Use GridsearchCV to determine Parameters

In [3]:
# from sklearn.model_selection import GridSearchCV
# gbm_parm_gs = {
#     'max_depth':[1] ,
#    'n_estimators': [50,100],
#    'learning_rate': [0.1,0.05,0.01]
# }
# gbm_model_gs = GradientBoostingClassifier()
# gbm_gsearch = GridSearchCV(estimator = gbm_model_gs, 
#                        param_grid = gbm_parm_gs, 
#                        scoring ='accuracy',
#                        cv = 3,
#                        n_jobs = 8,
#                        verbose=3,
#                        return_train_score=True
#                       )
# gbm_gsearch.fit(train_x_dis,train_y_dis)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  10 out of  18 | elapsed:  8.2min remaining:  6.6min
[Parallel(n_jobs=8)]: Done  18 out of  18 | elapsed: 15.1min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
                                                  presort=

In [4]:
# from sklearn.metrics import accuracy_score  
# print("Training dataset Accuracy")
# train_preds = gbm_gsearch.predict(train_x_dis)
# train_accuracy = accuracy_score(train_y_dis, train_preds) 
# print(train_accuracy)

Training dataset Accuracy
0.829


In [5]:
# print("Testing dataset Accuracy")
# test_preds = gbm_gsearch.predict(test_x_dis)
# test_accuracy = accuracy_score(test_y_dis, test_preds) 
# print(test_accuracy)

Testing dataset Accuracy
0.374


In [6]:
# gbm_gsearch.best_params_

{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 100}

### Use K-Fold to estimate the accuracy (41.92%)

In [7]:
gbm_baseline= GradientBoostingClassifier(n_estimators=100  , max_depth= 1,learning_rate=0.1)

In [8]:
# kfold = KFold(n_splits=10, shuffle=True)
# results = cross_val_score(gbm_baseline, X_dis,y_dis, cv=kfold)
# print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 41.92% (2.08%)


### Fit the final model and save it

In [9]:
t0 = time.time()
gbm_baseline.fit(X_dis,y_dis)
print("done in %0.3fs" % (time.time() - t0))

done in 414.886s


In [11]:
# joblib.dump(gbm_baseline,'gbm_baseline.m')#please change the path

['gbm_baseline.m']

In [ ]:
#joblib.load('C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/doc/gbm_baseline.m')